Cadiz stations

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import psutil
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Dropout, Flatten, Input
import tensorflow as tf
from bayes_opt import BayesianOptimization
from scipy.fftpack import fft
import warnings

# Set seeds
np.random.seed(42)
tf.random.set_seed(42)
warnings.filterwarnings('ignore')

# ============================================
# 1. Data Loading and Preprocessing
# ============================================
def load_and_preprocess_data(filepath):
    df = pd.read_csv(filepath, delimiter=",", low_memory=False)
    df.columns = df.columns.str.replace('"', '').str.strip()

    if 'TIME' not in df.columns:
        raise KeyError("The column 'TIME' does not exist in the dataset.")

    df['TIME'] = pd.to_datetime(df['TIME'], errors='coerce')
    df.dropna(subset=['TIME'], inplace=True)
    df.set_index('TIME', inplace=True)

    df = df.apply(pd.to_numeric, errors='coerce')
    df.dropna(axis=1, how='all', inplace=True)

    target_col = 'fenologia_h1'
    if target_col not in df.columns:
        raise KeyError(f"The target column '{target_col}' does not exist.")

    df[target_col].interpolate(method='linear', inplace=True)
    df[target_col] = df[target_col] - 1

    for i in range(1, 4):
        df[f'{target_col}_lag{i}'] = df[target_col].shift(i)

    for window in [3, 6]:
        df[f'{target_col}_roll_mean_{window}'] = df[target_col].rolling(window).mean()
        df[f'{target_col}_roll_std_{window}'] = df[target_col].rolling(window).std()

    # df['month'] = df.index.month
    # df['weekofyear'] = df.index.isocalendar().week
    # df['year'] = df.index.isocalendar().year
    # df['sin_week'] = np.sin(2 * np.pi * df['weekofyear'] / 52)
    # df['cos_week'] = np.cos(2 * np.pi * df['weekofyear'] / 52)
    # df['EMA_3'] = df[target_col].ewm(span=3, adjust=False).mean()
    # df['EMA_6'] = df[target_col].ewm(span=6, adjust=False).mean()
    # df['correlation_target_month'] = df[target_col].rolling(window=6).corr(df['month'])
    # df['correlation_target_week'] = df[target_col].rolling(window=6).corr(df['weekofyear'])

    # fft_values = fft(df[target_col].dropna().values)
    # fft_real = np.real(fft_values)[:len(df[target_col])]
    # fft_imag = np.imag(fft_values)[:len(df[target_col])]
    # df['fft_real'] = np.concatenate([fft_real, np.nan * np.ones(len(df) - len(fft_real))])
    # df['fft_imag'] = np.concatenate([fft_imag, np.nan * np.ones(len(df) - len(fft_imag))])

    df.fillna(df.median(), inplace=True)
    return df, target_col

# ============================================
# 2. Feature Scaling
# ============================================
def feature_scaling(df, target_col):
    X = df.drop(columns=[target_col])
    y = df[target_col]

    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = X_scaled.reshape((X_scaled.shape[0], X_scaled.shape[1], 1))  # 3D for CNN
    return X_scaled, y, scaler

# ============================================
# 3. Build TCN Model (Dilated CNN)
# ============================================
def build_tcn_model(input_shape, num_classes=4, num_filters=64, kernel_size=3, dropout_rate=0.2, dilation_rate=2):
    model = Sequential()
    
    # Add dilated convolutions
    model.add(Conv1D(filters=num_filters, kernel_size=kernel_size, dilation_rate=dilation_rate, 
                     activation='relu', input_shape=input_shape))
    model.add(Dropout(dropout_rate))
    
    # Add more dilated convolutions if needed
    model.add(Conv1D(filters=num_filters, kernel_size=kernel_size, dilation_rate=dilation_rate, activation='relu'))
    model.add(Dropout(dropout_rate))

    # Flatten and add Dense layers
    model.add(Flatten())
    model.add(Dense(num_filters // 2, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# ============================================
# 4. Bayesian Optimization Objective
# ============================================
def objective_function(num_filters, dropout_rate, epochs, batch_size, X_train, y_train):
    model = build_tcn_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_filters=int(num_filters),
        dropout_rate=dropout_rate
    )
    history = model.fit(
        X_train, y_train,
        epochs=int(epochs),
        batch_size=int(batch_size),
        validation_split=0.2,
        verbose=0
    )
    return np.max(history.history['val_accuracy'])

# ============================================
# 5. Optimize Hyperparameters
# ============================================
def optimize_hyperparameters(X_train, y_train):
    pbounds = {
        'num_filters': (32, 128),
        'dropout_rate': (0.1, 0.5),
        'epochs': (50, 100),
        'batch_size': (16, 64)
    }

    optimizer = BayesianOptimization(
        f=lambda num_filters, dropout_rate, epochs, batch_size: objective_function(
            num_filters, dropout_rate, epochs, batch_size, X_train, y_train
        ),
        pbounds=pbounds,
        random_state=42,
        verbose=2
    )

    optimizer.maximize(init_points=5, n_iter=15)
    return optimizer.max['params']

# ============================================
# 6. Train & Evaluate Model
# ============================================
def train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params):
    model = build_tcn_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_filters=int(best_params['num_filters']),
        dropout_rate=float(best_params['dropout_rate'])
    )

    start_train = time.time()
    history = model.fit(
        X_train, y_train,
        epochs=int(best_params['epochs']),
        batch_size=int(best_params['batch_size']),
        validation_split=0.2,
        verbose=1
    )
    training_time = time.time() - start_train

    start_infer = time.time()
    y_pred = model.predict(X_test).argmax(axis=1)
    inference_time = (time.time() - start_infer) / len(X_test)

    # Calculate total testing time
    testing_time = time.time() - start_infer

    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    print("\n===== Evaluation =====")
    print(f"Accuracy: {acc:.4f}")
    print("Report:\n", classification_report(y_test, y_pred, digits=4))
    print("Confusion Matrix:\n", cm)

    model.save("temp_model.h5", include_optimizer=False)
    model_size = os.path.getsize("temp_model.h5") / (1024 ** 2)
    os.remove("temp_model.h5")

    total_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
    ram_usage = psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2)

    print("\n===== Model Metrics =====")
    print(f"Training Time: {training_time:.2f}s")
    print(f"Inference/sample: {inference_time:.6f}s")
    print(f"Testing Time: {testing_time:.2f}s")
    print(f"Model Size: {model_size:.2f} MB")
    print(f"Trainable Params: {total_params:,}")
    print(f"RAM Usage: {ram_usage:.2f} MB")

    return model, history

# ============================================
# 7. Main
# ============================================
def main():
    train_path =  r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H1\Cadiz_train.csv"
    test_path =  r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H1\Cadiz_test.csv"

    df_train, target_col = load_and_preprocess_data(train_path)
    X_train, y_train, _ = feature_scaling(df_train, target_col)

    df_test, _ = load_and_preprocess_data(test_path)
    X_test, y_test, _ = feature_scaling(df_test, target_col)

    print("\n Optimizing Hyperparameters...")
    best_params = optimize_hyperparameters(X_train, y_train)
    print("\n Best Params:", best_params)

    print("\n Training Final TCN Model...")
    model, history = train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params)

if __name__ == "__main__":
    main()



 Optimizing Hyperparameters...
|   iter    |  target   | batch_... | dropou... |  epochs   | num_fi... |
-------------------------------------------------------------------------
| 1         | 1.0       | 33.98     | 0.4803    | 86.6      | 89.47     |
| 2         | 1.0       | 23.49     | 0.1624    | 52.9      | 115.2     |
| 3         | 1.0       | 44.85     | 0.3832    | 51.03     | 125.1     |
| 4         | 0.9692    | 55.96     | 0.1849    | 59.09     | 49.61     |
| 5         | 1.0       | 30.6      | 0.3099    | 71.6      | 59.96     |
| 6         | 1.0       | 31.85     | 0.4144    | 54.7      | 125.6     |
| 7         | 1.0       | 16.22     | 0.1008    | 98.87     | 57.36     |
| 8         | 1.0       | 24.08     | 0.3354    | 54.27     | 116.3     |
| 9         | 1.0       | 62.74     | 0.2653    | 99.33     | 126.9     |
| 10        | 1.0       | 21.2      | 0.3698    | 99.6      | 128.0     |
| 11        | 1.0       | 16.02     | 0.4903    | 58.02     | 77.58     |
| 12  


===== Evaluation =====
Accuracy: 0.9655
Report:
               precision    recall  f1-score   support

           0     0.8000    0.7500    0.7742        16
           1     0.9296    0.9565    0.9429        69
           2     1.0000    0.9927    0.9963       137
           3     1.0000    1.0000    1.0000        10

    accuracy                         0.9655       232
   macro avg     0.9324    0.9248    0.9283       232
weighted avg     0.9653    0.9655    0.9653       232

Confusion Matrix:
 [[ 12   4   0   0]
 [  3  66   0   0]
 [  0   1 136   0]
 [  0   0   0  10]]

===== Model Metrics =====
Training Time: 22.84s
Inference/sample: 0.002544s
Testing Time: 0.59s
Model Size: 0.26 MB
Trainable Params: 59,676
RAM Usage: 953.00 MB


Cordoba station

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import psutil
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Dropout, Flatten, Input
import tensorflow as tf
from bayes_opt import BayesianOptimization
from scipy.fftpack import fft
import warnings

# Set seeds
np.random.seed(42)
tf.random.set_seed(42)
warnings.filterwarnings('ignore')

# ============================================
# 1. Data Loading and Preprocessing
# ============================================
def load_and_preprocess_data(filepath):
    df = pd.read_csv(filepath, delimiter=",", low_memory=False)
    df.columns = df.columns.str.replace('"', '').str.strip()

    if 'TIME' not in df.columns:
        raise KeyError("The column 'TIME' does not exist in the dataset.")

    df['TIME'] = pd.to_datetime(df['TIME'], errors='coerce')
    df.dropna(subset=['TIME'], inplace=True)
    df.set_index('TIME', inplace=True)

    df = df.apply(pd.to_numeric, errors='coerce')
    df.dropna(axis=1, how='all', inplace=True)

    target_col = 'fenologia_h1'
    if target_col not in df.columns:
        raise KeyError(f"The target column '{target_col}' does not exist.")

    df[target_col].interpolate(method='linear', inplace=True)
    df[target_col] = df[target_col] - 1

    for i in range(1, 4):
        df[f'{target_col}_lag{i}'] = df[target_col].shift(i)

    for window in [3, 6]:
        df[f'{target_col}_roll_mean_{window}'] = df[target_col].rolling(window).mean()
        df[f'{target_col}_roll_std_{window}'] = df[target_col].rolling(window).std()

    # df['month'] = df.index.month
    # df['weekofyear'] = df.index.isocalendar().week
    # df['year'] = df.index.isocalendar().year
    # df['sin_week'] = np.sin(2 * np.pi * df['weekofyear'] / 52)
    # df['cos_week'] = np.cos(2 * np.pi * df['weekofyear'] / 52)
    # df['EMA_3'] = df[target_col].ewm(span=3, adjust=False).mean()
    # df['EMA_6'] = df[target_col].ewm(span=6, adjust=False).mean()
    # df['correlation_target_month'] = df[target_col].rolling(window=6).corr(df['month'])
    # df['correlation_target_week'] = df[target_col].rolling(window=6).corr(df['weekofyear'])

    # fft_values = fft(df[target_col].dropna().values)
    # fft_real = np.real(fft_values)[:len(df[target_col])]
    # fft_imag = np.imag(fft_values)[:len(df[target_col])]
    # df['fft_real'] = np.concatenate([fft_real, np.nan * np.ones(len(df) - len(fft_real))])
    # df['fft_imag'] = np.concatenate([fft_imag, np.nan * np.ones(len(df) - len(fft_imag))])

    df.fillna(df.median(), inplace=True)
    return df, target_col

# ============================================
# 2. Feature Scaling
# ============================================
def feature_scaling(df, target_col):
    X = df.drop(columns=[target_col])
    y = df[target_col]

    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = X_scaled.reshape((X_scaled.shape[0], X_scaled.shape[1], 1))  # 3D for CNN
    return X_scaled, y, scaler

# ============================================
# 3. Build TCN Model (Dilated CNN)
# ============================================
def build_tcn_model(input_shape, num_classes=4, num_filters=64, kernel_size=3, dropout_rate=0.2, dilation_rate=2):
    model = Sequential()
    
    # Add dilated convolutions
    model.add(Conv1D(filters=num_filters, kernel_size=kernel_size, dilation_rate=dilation_rate, 
                     activation='relu', input_shape=input_shape))
    model.add(Dropout(dropout_rate))
    
    # Add more dilated convolutions if needed
    model.add(Conv1D(filters=num_filters, kernel_size=kernel_size, dilation_rate=dilation_rate, activation='relu'))
    model.add(Dropout(dropout_rate))

    # Flatten and add Dense layers
    model.add(Flatten())
    model.add(Dense(num_filters // 2, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# ============================================
# 4. Bayesian Optimization Objective
# ============================================
def objective_function(num_filters, dropout_rate, epochs, batch_size, X_train, y_train):
    model = build_tcn_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_filters=int(num_filters),
        dropout_rate=dropout_rate
    )
    history = model.fit(
        X_train, y_train,
        epochs=int(epochs),
        batch_size=int(batch_size),
        validation_split=0.2,
        verbose=0
    )
    return np.max(history.history['val_accuracy'])

# ============================================
# 5. Optimize Hyperparameters
# ============================================
def optimize_hyperparameters(X_train, y_train):
    pbounds = {
        'num_filters': (32, 128),
        'dropout_rate': (0.1, 0.5),
        'epochs': (50, 100),
        'batch_size': (16, 64)
    }

    optimizer = BayesianOptimization(
        f=lambda num_filters, dropout_rate, epochs, batch_size: objective_function(
            num_filters, dropout_rate, epochs, batch_size, X_train, y_train
        ),
        pbounds=pbounds,
        random_state=42,
        verbose=2
    )

    optimizer.maximize(init_points=5, n_iter=15)
    return optimizer.max['params']

# ============================================
# 6. Train & Evaluate Model
# ============================================
def train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params):
    model = build_tcn_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_filters=int(best_params['num_filters']),
        dropout_rate=float(best_params['dropout_rate'])
    )

    start_train = time.time()
    history = model.fit(
        X_train, y_train,
        epochs=int(best_params['epochs']),
        batch_size=int(best_params['batch_size']),
        validation_split=0.2,
        verbose=1
    )
    training_time = time.time() - start_train

    start_infer = time.time()
    y_pred = model.predict(X_test).argmax(axis=1)
    inference_time = (time.time() - start_infer) / len(X_test)

    # Calculate total testing time
    testing_time = time.time() - start_infer

    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    print("\n===== Evaluation =====")
    print(f"Accuracy: {acc:.4f}")
    print("Report:\n", classification_report(y_test, y_pred, digits=4))
    print("Confusion Matrix:\n", cm)

    model.save("temp_model.h5", include_optimizer=False)
    model_size = os.path.getsize("temp_model.h5") / (1024 ** 2)
    os.remove("temp_model.h5")

    total_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
    ram_usage = psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2)

    print("\n===== Model Metrics =====")
    print(f"Training Time: {training_time:.2f}s")
    print(f"Inference/sample: {inference_time:.6f}s")
    print(f"Testing Time: {testing_time:.2f}s")
    print(f"Model Size: {model_size:.2f} MB")
    print(f"Trainable Params: {total_params:,}")
    print(f"RAM Usage: {ram_usage:.2f} MB")

    return model, history

# ============================================
# 7. Main
# ============================================
def main():
    train_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H1\Cordoba_train.csv"
    test_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H1\Cordoba_test.csv"

    df_train, target_col = load_and_preprocess_data(train_path)
    X_train, y_train, _ = feature_scaling(df_train, target_col)

    df_test, _ = load_and_preprocess_data(test_path)
    X_test, y_test, _ = feature_scaling(df_test, target_col)

    print("\n Optimizing Hyperparameters...")
    best_params = optimize_hyperparameters(X_train, y_train)
    print("\n Best Params:", best_params)

    print("\n Training Final TCN Model...")
    model, history = train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params)

if __name__ == "__main__":
    main()



 Optimizing Hyperparameters...
|   iter    |  target   | batch_... | dropou... |  epochs   | num_fi... |
-------------------------------------------------------------------------
| 1         | 1.0       | 33.98     | 0.4803    | 86.6      | 89.47     |
| 2         | 0.9839    | 23.49     | 0.1624    | 52.9      | 115.2     |
| 3         | 0.9839    | 44.85     | 0.3832    | 51.03     | 125.1     |
| 4         | 0.9677    | 55.96     | 0.1849    | 59.09     | 49.61     |
| 5         | 0.9839    | 30.6      | 0.3099    | 71.6      | 59.96     |
| 6         | 1.0       | 19.66     | 0.4683    | 98.68     | 90.17     |
| 7         | 0.9677    | 49.31     | 0.2359    | 100.0     | 117.8     |
| 8         | 1.0       | 33.12     | 0.3515    | 85.97     | 89.07     |
| 9         | 1.0       | 34.41     | 0.3111    | 99.28     | 68.82     |
| 10        | 0.9839    | 18.2      | 0.4567    | 99.88     | 32.67     |
| 11        | 0.9839    | 58.85     | 0.3279    | 99.81     | 73.71     |
| 12  


===== Evaluation =====
Accuracy: 0.8851
Report:
               precision    recall  f1-score   support

           0     0.7143    0.2941    0.4167        17
           1     0.7241    0.9333    0.8155        45
           2     0.9897    0.9505    0.9697       101
           3     0.9167    1.0000    0.9565        11

    accuracy                         0.8851       174
   macro avg     0.8362    0.7945    0.7896       174
weighted avg     0.8895    0.8851    0.8750       174

Confusion Matrix:
 [[ 5 12  0  0]
 [ 2 42  1  0]
 [ 0  4 96  1]
 [ 0  0  0 11]]

===== Model Metrics =====
Training Time: 18.13s
Inference/sample: 0.001959s
Testing Time: 0.34s
Model Size: 0.26 MB
Trainable Params: 59,676
RAM Usage: 1277.34 MB


Jaen Station

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import psutil
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Dropout, Flatten, Input
import tensorflow as tf
from bayes_opt import BayesianOptimization
from scipy.fftpack import fft
import warnings

# Set seeds
np.random.seed(42)
tf.random.set_seed(42)
warnings.filterwarnings('ignore')

# ============================================
# 1. Data Loading and Preprocessing
# ============================================
def load_and_preprocess_data(filepath):
    df = pd.read_csv(filepath, delimiter=",", low_memory=False)
    df.columns = df.columns.str.replace('"', '').str.strip()

    if 'TIME' not in df.columns:
        raise KeyError("The column 'TIME' does not exist in the dataset.")

    df['TIME'] = pd.to_datetime(df['TIME'], errors='coerce')
    df.dropna(subset=['TIME'], inplace=True)
    df.set_index('TIME', inplace=True)

    df = df.apply(pd.to_numeric, errors='coerce')
    df.dropna(axis=1, how='all', inplace=True)

    target_col = 'fenologia_h1'
    if target_col not in df.columns:
        raise KeyError(f"The target column '{target_col}' does not exist.")

    df[target_col].interpolate(method='linear', inplace=True)
    df[target_col] = df[target_col] - 1

    for i in range(1, 4):
        df[f'{target_col}_lag{i}'] = df[target_col].shift(i)

    for window in [3, 6]:
        df[f'{target_col}_roll_mean_{window}'] = df[target_col].rolling(window).mean()
        df[f'{target_col}_roll_std_{window}'] = df[target_col].rolling(window).std()

    # df['month'] = df.index.month
    # df['weekofyear'] = df.index.isocalendar().week
    # df['year'] = df.index.isocalendar().year
    # df['sin_week'] = np.sin(2 * np.pi * df['weekofyear'] / 52)
    # df['cos_week'] = np.cos(2 * np.pi * df['weekofyear'] / 52)
    # df['EMA_3'] = df[target_col].ewm(span=3, adjust=False).mean()
    # df['EMA_6'] = df[target_col].ewm(span=6, adjust=False).mean()
    # df['correlation_target_month'] = df[target_col].rolling(window=6).corr(df['month'])
    # df['correlation_target_week'] = df[target_col].rolling(window=6).corr(df['weekofyear'])

    # fft_values = fft(df[target_col].dropna().values)
    # fft_real = np.real(fft_values)[:len(df[target_col])]
    # fft_imag = np.imag(fft_values)[:len(df[target_col])]
    # df['fft_real'] = np.concatenate([fft_real, np.nan * np.ones(len(df) - len(fft_real))])
    # df['fft_imag'] = np.concatenate([fft_imag, np.nan * np.ones(len(df) - len(fft_imag))])

    df.fillna(df.median(), inplace=True)
    return df, target_col

# ============================================
# 2. Feature Scaling
# ============================================
def feature_scaling(df, target_col):
    X = df.drop(columns=[target_col])
    y = df[target_col]

    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = X_scaled.reshape((X_scaled.shape[0], X_scaled.shape[1], 1))  # 3D for CNN
    return X_scaled, y, scaler

# ============================================
# 3. Build TCN Model (Dilated CNN)
# ============================================
def build_tcn_model(input_shape, num_classes=4, num_filters=64, kernel_size=3, dropout_rate=0.2, dilation_rate=2):
    model = Sequential()
    
    # Add dilated convolutions
    model.add(Conv1D(filters=num_filters, kernel_size=kernel_size, dilation_rate=dilation_rate, 
                     activation='relu', input_shape=input_shape))
    model.add(Dropout(dropout_rate))
    
    # Add more dilated convolutions if needed
    model.add(Conv1D(filters=num_filters, kernel_size=kernel_size, dilation_rate=dilation_rate, activation='relu'))
    model.add(Dropout(dropout_rate))

    # Flatten and add Dense layers
    model.add(Flatten())
    model.add(Dense(num_filters // 2, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# ============================================
# 4. Bayesian Optimization Objective
# ============================================
def objective_function(num_filters, dropout_rate, epochs, batch_size, X_train, y_train):
    model = build_tcn_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_filters=int(num_filters),
        dropout_rate=dropout_rate
    )
    history = model.fit(
        X_train, y_train,
        epochs=int(epochs),
        batch_size=int(batch_size),
        validation_split=0.2,
        verbose=0
    )
    return np.max(history.history['val_accuracy'])

# ============================================
# 5. Optimize Hyperparameters
# ============================================
def optimize_hyperparameters(X_train, y_train):
    pbounds = {
        'num_filters': (32, 128),
        'dropout_rate': (0.1, 0.5),
        'epochs': (50, 100),
        'batch_size': (16, 64)
    }

    optimizer = BayesianOptimization(
        f=lambda num_filters, dropout_rate, epochs, batch_size: objective_function(
            num_filters, dropout_rate, epochs, batch_size, X_train, y_train
        ),
        pbounds=pbounds,
        random_state=42,
        verbose=2
    )

    optimizer.maximize(init_points=5, n_iter=15)
    return optimizer.max['params']

# ============================================
# 6. Train & Evaluate Model
# ============================================
def train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params):
    model = build_tcn_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_filters=int(best_params['num_filters']),
        dropout_rate=float(best_params['dropout_rate'])
    )

    start_train = time.time()
    history = model.fit(
        X_train, y_train,
        epochs=int(best_params['epochs']),
        batch_size=int(best_params['batch_size']),
        validation_split=0.2,
        verbose=1
    )
    training_time = time.time() - start_train

    start_infer = time.time()
    y_pred = model.predict(X_test).argmax(axis=1)
    inference_time = (time.time() - start_infer) / len(X_test)

    # Calculate total testing time
    testing_time = time.time() - start_infer

    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    print("\n===== Evaluation =====")
    print(f"Accuracy: {acc:.4f}")
    print("Report:\n", classification_report(y_test, y_pred, digits=4))
    print("Confusion Matrix:\n", cm)

    model.save("temp_model.h5", include_optimizer=False)
    model_size = os.path.getsize("temp_model.h5") / (1024 ** 2)
    os.remove("temp_model.h5")

    total_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
    ram_usage = psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2)

    print("\n===== Model Metrics =====")
    print(f"Training Time: {training_time:.2f}s")
    print(f"Inference/sample: {inference_time:.6f}s")
    print(f"Testing Time: {testing_time:.2f}s")
    print(f"Model Size: {model_size:.2f} MB")
    print(f"Trainable Params: {total_params:,}")
    print(f"RAM Usage: {ram_usage:.2f} MB")

    return model, history

# ============================================
# 7. Main
# ============================================
def main():
    train_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H1\jaen_train.csv"
    test_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H1\jaen_test.csv"

    df_train, target_col = load_and_preprocess_data(train_path)
    X_train, y_train, _ = feature_scaling(df_train, target_col)

    df_test, _ = load_and_preprocess_data(test_path)
    X_test, y_test, _ = feature_scaling(df_test, target_col)

    print("\n Optimizing Hyperparameters...")
    best_params = optimize_hyperparameters(X_train, y_train)
    print("\n Best Params:", best_params)

    print("\n Training Final TCN Model...")
    model, history = train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params)

if __name__ == "__main__":
    main()



 Optimizing Hyperparameters...
|   iter    |  target   | batch_... | dropou... |  epochs   | num_fi... |
-------------------------------------------------------------------------
| 1         | 1.0       | 33.98     | 0.4803    | 86.6      | 89.47     |
| 2         | 1.0       | 23.49     | 0.1624    | 52.9      | 115.2     |
| 3         | 0.96      | 44.85     | 0.3832    | 51.03     | 125.1     |
| 4         | 0.96      | 55.96     | 0.1849    | 59.09     | 49.61     |
| 5         | 1.0       | 30.6      | 0.3099    | 71.6      | 59.96     |
| 6         | 1.0       | 22.97     | 0.3185    | 53.09     | 116.6     |
| 7         | 1.0       | 22.24     | 0.4959    | 71.26     | 80.54     |
| 8         | 1.0       | 19.19     | 0.136     | 92.37     | 66.74     |
| 9         | 1.0       | 16.28     | 0.4313    | 75.57     | 111.3     |
| 10        | 1.0       | 16.86     | 0.2711    | 88.25     | 36.35     |
| 11        | 1.0       | 18.27     | 0.3674    | 99.38     | 128.0     |
| 12  

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step



===== Evaluation =====
Accuracy: 0.6957
Report:
               precision    recall  f1-score   support

           0     0.5294    0.6923    0.6000        13
           1     0.8966    0.5200    0.6582        50
           2     0.8692    0.7686    0.8158       121
           3     0.0000    0.0000    0.0000         0

    accuracy                         0.6957       184
   macro avg     0.5738    0.4952    0.5185       184
weighted avg     0.8526    0.6957    0.7577       184

Confusion Matrix:
 [[ 9  3  0  1]
 [ 8 26 14  2]
 [ 0  0 93 28]
 [ 0  0  0  0]]

===== Model Metrics =====
Training Time: 10.79s
Inference/sample: 0.001549s
Testing Time: 0.28s
Model Size: 0.26 MB
Trainable Params: 59,676
RAM Usage: 1651.69 MB


Sevilla station

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import psutil
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Dropout, Flatten, Input
import tensorflow as tf
from bayes_opt import BayesianOptimization
from scipy.fftpack import fft
import warnings

# Set seeds
np.random.seed(42)
tf.random.set_seed(42)
warnings.filterwarnings('ignore')

# ============================================
# 1. Data Loading and Preprocessing
# ============================================
def load_and_preprocess_data(filepath):
    df = pd.read_csv(filepath, delimiter=",", low_memory=False)
    df.columns = df.columns.str.replace('"', '').str.strip()

    if 'TIME' not in df.columns:
        raise KeyError("The column 'TIME' does not exist in the dataset.")

    df['TIME'] = pd.to_datetime(df['TIME'], errors='coerce')
    df.dropna(subset=['TIME'], inplace=True)
    df.set_index('TIME', inplace=True)

    df = df.apply(pd.to_numeric, errors='coerce')
    df.dropna(axis=1, how='all', inplace=True)

    target_col = 'fenologia_h1'
    if target_col not in df.columns:
        raise KeyError(f"The target column '{target_col}' does not exist.")

    df[target_col].interpolate(method='linear', inplace=True)
    df[target_col] = df[target_col] - 1

    for i in range(1, 4):
        df[f'{target_col}_lag{i}'] = df[target_col].shift(i)

    for window in [3, 6]:
        df[f'{target_col}_roll_mean_{window}'] = df[target_col].rolling(window).mean()
        df[f'{target_col}_roll_std_{window}'] = df[target_col].rolling(window).std()

    # df['month'] = df.index.month
    # df['weekofyear'] = df.index.isocalendar().week
    # df['year'] = df.index.isocalendar().year
    # df['sin_week'] = np.sin(2 * np.pi * df['weekofyear'] / 52)
    # df['cos_week'] = np.cos(2 * np.pi * df['weekofyear'] / 52)
    # df['EMA_3'] = df[target_col].ewm(span=3, adjust=False).mean()
    # df['EMA_6'] = df[target_col].ewm(span=6, adjust=False).mean()
    # df['correlation_target_month'] = df[target_col].rolling(window=6).corr(df['month'])
    # df['correlation_target_week'] = df[target_col].rolling(window=6).corr(df['weekofyear'])

    # fft_values = fft(df[target_col].dropna().values)
    # fft_real = np.real(fft_values)[:len(df[target_col])]
    # fft_imag = np.imag(fft_values)[:len(df[target_col])]
    # df['fft_real'] = np.concatenate([fft_real, np.nan * np.ones(len(df) - len(fft_real))])
    # df['fft_imag'] = np.concatenate([fft_imag, np.nan * np.ones(len(df) - len(fft_imag))])

    df.fillna(df.median(), inplace=True)
    return df, target_col

# ============================================
# 2. Feature Scaling
# ============================================
def feature_scaling(df, target_col):
    X = df.drop(columns=[target_col])
    y = df[target_col]

    scaler = MinMaxScaler()
    X_scaled = scaler.fit_transform(X)
    X_scaled = X_scaled.reshape((X_scaled.shape[0], X_scaled.shape[1], 1))  # 3D for CNN
    return X_scaled, y, scaler

# ============================================
# 3. Build TCN Model (Dilated CNN)
# ============================================
def build_tcn_model(input_shape, num_classes=4, num_filters=64, kernel_size=3, dropout_rate=0.2, dilation_rate=2):
    model = Sequential()
    
    # Add dilated convolutions
    model.add(Conv1D(filters=num_filters, kernel_size=kernel_size, dilation_rate=dilation_rate, 
                     activation='relu', input_shape=input_shape))
    model.add(Dropout(dropout_rate))
    
    # Add more dilated convolutions if needed
    model.add(Conv1D(filters=num_filters, kernel_size=kernel_size, dilation_rate=dilation_rate, activation='relu'))
    model.add(Dropout(dropout_rate))

    # Flatten and add Dense layers
    model.add(Flatten())
    model.add(Dense(num_filters // 2, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# ============================================
# 4. Bayesian Optimization Objective
# ============================================
def objective_function(num_filters, dropout_rate, epochs, batch_size, X_train, y_train):
    model = build_tcn_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_filters=int(num_filters),
        dropout_rate=dropout_rate
    )
    history = model.fit(
        X_train, y_train,
        epochs=int(epochs),
        batch_size=int(batch_size),
        validation_split=0.2,
        verbose=0
    )
    return np.max(history.history['val_accuracy'])

# ============================================
# 5. Optimize Hyperparameters
# ============================================
def optimize_hyperparameters(X_train, y_train):
    pbounds = {
        'num_filters': (32, 128),
        'dropout_rate': (0.1, 0.5),
        'epochs': (50, 100),
        'batch_size': (16, 64)
    }

    optimizer = BayesianOptimization(
        f=lambda num_filters, dropout_rate, epochs, batch_size: objective_function(
            num_filters, dropout_rate, epochs, batch_size, X_train, y_train
        ),
        pbounds=pbounds,
        random_state=42,
        verbose=2
    )

    optimizer.maximize(init_points=5, n_iter=15)
    return optimizer.max['params']

# ============================================
# 6. Train & Evaluate Model
# ============================================
def train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params):
    model = build_tcn_model(
        input_shape=(X_train.shape[1], X_train.shape[2]),
        num_classes=4,
        num_filters=int(best_params['num_filters']),
        dropout_rate=float(best_params['dropout_rate'])
    )

    start_train = time.time()
    history = model.fit(
        X_train, y_train,
        epochs=int(best_params['epochs']),
        batch_size=int(best_params['batch_size']),
        validation_split=0.2,
        verbose=1
    )
    training_time = time.time() - start_train

    start_infer = time.time()
    y_pred = model.predict(X_test).argmax(axis=1)
    inference_time = (time.time() - start_infer) / len(X_test)

    # Calculate total testing time
    testing_time = time.time() - start_infer

    acc = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)

    print("\n===== Evaluation =====")
    print(f"Accuracy: {acc:.4f}")
    print("Report:\n", classification_report(y_test, y_pred, digits=4))
    print("Confusion Matrix:\n", cm)

    model.save("temp_model.h5", include_optimizer=False)
    model_size = os.path.getsize("temp_model.h5") / (1024 ** 2)
    os.remove("temp_model.h5")

    total_params = np.sum([np.prod(v.shape) for v in model.trainable_weights])
    ram_usage = psutil.Process(os.getpid()).memory_info().rss / (1024 ** 2)

    print("\n===== Model Metrics =====")
    print(f"Training Time: {training_time:.2f}s")
    print(f"Inference/sample: {inference_time:.6f}s")
    print(f"Testing Time: {testing_time:.2f}s")
    print(f"Model Size: {model_size:.2f} MB")
    print(f"Trainable Params: {total_params:,}")
    print(f"RAM Usage: {ram_usage:.2f} MB")

    return model, history

# ============================================
# 7. Main
# ============================================
def main():
    train_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H1\sevilla_train.csv"
    test_path = r"E:\Abroad period research\Phenology datasets\PHENOLOGY_H1\sevilla_test.csv"

    df_train, target_col = load_and_preprocess_data(train_path)
    X_train, y_train, _ = feature_scaling(df_train, target_col)

    df_test, _ = load_and_preprocess_data(test_path)
    X_test, y_test, _ = feature_scaling(df_test, target_col)

    print("\n Optimizing Hyperparameters...")
    best_params = optimize_hyperparameters(X_train, y_train)
    print("\n Best Params:", best_params)

    print("\n Training Final TCN Model...")
    model, history = train_and_evaluate_model(X_train, X_test, y_train, y_test, best_params)

if __name__ == "__main__":
    main()



 Optimizing Hyperparameters...
|   iter    |  target   | batch_... | dropou... |  epochs   | num_fi... |
-------------------------------------------------------------------------
| 1         | 1.0       | 33.98     | 0.4803    | 86.6      | 89.47     |
| 2         | 1.0       | 23.49     | 0.1624    | 52.9      | 115.2     |
| 3         | 1.0       | 44.85     | 0.3832    | 51.03     | 125.1     |
| 4         | 0.9857    | 55.96     | 0.1849    | 59.09     | 49.61     |
| 5         | 1.0       | 30.6      | 0.3099    | 71.6      | 59.96     |
| 6         | 1.0       | 31.85     | 0.4144    | 54.7      | 125.6     |
| 7         | 1.0       | 16.22     | 0.1008    | 98.87     | 57.36     |
| 8         | 1.0       | 24.08     | 0.3354    | 54.27     | 116.3     |
| 9         | 1.0       | 62.74     | 0.2653    | 99.33     | 126.9     |
| 10        | 1.0       | 21.2      | 0.3698    | 99.6      | 128.0     |
| 11        | 1.0       | 16.02     | 0.4903    | 58.02     | 77.58     |
| 12  

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step



===== Evaluation =====
Accuracy: 0.9238
Report:
               precision    recall  f1-score   support

           0     0.5000    0.3000    0.3750        10
           1     0.9114    0.9474    0.9290        76
           2     0.9840    0.9609    0.9723       128
           3     0.6154    0.8889    0.7273         9

    accuracy                         0.9238       223
   macro avg     0.7527    0.7743    0.7509       223
weighted avg     0.9227    0.9238    0.9209       223

Confusion Matrix:
 [[  3   7   0   0]
 [  3  72   1   0]
 [  0   0 123   5]
 [  0   0   1   8]]

===== Model Metrics =====
Training Time: 12.66s
Inference/sample: 0.000942s
Testing Time: 0.21s
Model Size: 0.26 MB
Trainable Params: 59,676
RAM Usage: 1930.36 MB
